In [1]:
import pandas as pd
import numpy as np

In [2]:
# Надо бы посмотреть чего нового
pd.__version__

'0.23.0'

In [3]:
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)
%timeit x + y

4.19 ms ± 495 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

263 ms ± 4.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
# однако при вычисление составных выражений данная конструкция оказывается менее эффективной
# Например чтобы вычислить mask = (x > 0.5) & (y < 0.5) выполняются следующие действия
tmp1 = (x > 0.5)    # поскольку numpy вычисляет каждое подвыражение отдельно
tmp2 = (y < 0.5)

mask = tmp1 & tmp2
# Для каждого промежуточного итога явным образом выделяется память
# что при больших массивах x и y может привезти к большим накладным расходам

In [6]:
# библиотека Numexpr позволяет вычислять подобные составные выражения поэлементно, 
# не требуя выделения памяти под промежуточные массивы целиком.
import numexpr       # Принимает на входе строку, содержащую выражение в стиле библиотеки numpy
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)
# Инструменты eval() query(), идеологически схожа и используют пакеи numexpr

True

## pandas.eval()

In [7]:
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols)) for i in range(4))

In [8]:
%timeit df1 + df2 + df3 + df4

89.1 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
# версия этого выражения работает на 50% быстрее
%timeit pd.eval('df1 + df2 + df3 + df4')

44 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Поддерживаемые функцией pd.eval() операции

In [10]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3))) for i in range(5)) 

In [11]:
# Функция eval поддерживает все арифметические операторы
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

In [12]:
# Операторы сравнения 
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [13]:
# Побитовые операторы
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [14]:
# Побитовые операторы
# Кроме того, она допускает использование литералов or и and в булевых выражениях
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

In [15]:
# Поддержка доступа к атрибутам объектов с помощью синтаксиса obj.attr и к индексам по средством obj[index]
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

#### Использование DataFrame.eval() для выполнения операций по столбцам

In [16]:
# у объекта DataFrame есть метод eval(), который работает как и высокоуровневая функция pd.eval()
# Но на столбцы можно ссылать по имени
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval('(df.A + df.B) / (df.C - 1)')
result3 = df.eval('(A + B) / (C - 1)')
print(np.allclose(result1, result2), np.allclose(result1, result3))

True True


In [17]:
# Метод DataFrame можно использовать, например для создания нового столбца
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


In [18]:
# Также можно модифицировать значение любого уже существующего столбца
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


#### Локальные переменные в методе DataFrame.eval()

In [19]:
# дополнительный синтаксис для работы с переменными
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')  # @ отмечает имя переменной, а не имя столбца
np.allclose(result1, result2)          # синтаксис @ поддерживается только DatFrame.eval(), но не функцией pd.eval()

True

## pandas.query()

In [22]:
# с помощью обычной индексации или pd.eval() при желании мы можем задать условия по строкам
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
%timeit result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
result2= pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)
# но с помощью метода DataFrame.eval() можно обращаться только к столбцам

24.4 ms ± 383 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


True

In [23]:
# Поэтому у объектов DataFrame есть еще один метод для вычсиления строк - DataFrame.query()

%timeit result3 = df.query('A < 0.5 and B < 0.5')
result3 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result3)
# такой код - быстрее и понятнее читается

1.75 ms ± 4.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


True

In [24]:
# Метод, также позволяет использовать флаг @ для локальных переменных
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

In [25]:
# традиционный метод работает быстрее на маленьких массивах
# Преимущество метода eval|query заключается в экономии оператичной памяти и иногда - в более панятном синтаксисе